In [ ]:
!pip install simpletransformers

In [ ]:
import numpy as np
import ast
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel, NERArgs
import logging

In [ ]:
df = pd.read_excel("/content/Final_ner_train.xlsx")
features = df
features = features.sample(frac=1, random_state=42)
split_index = int(len(features) * 0.8)
train_data = features[:split_index]
valid_data = features[split_index:]

In [ ]:
#trainset
train_lst = []
valid_lst = []
for k in train_data.index:
  id = train_data['ID'][k].replace('[','').replace(']','').split(",")
  token = train_data['Tokenizer'][k]
  token = ast.literal_eval(token)
  for i in range(len(id)):
    train_lst.append([k,token[i],str(int(id[i]))])

In [ ]:
# validset
for k in valid_data.index:
  id = valid_data['ID'][k].replace('[','').replace(']','').split(",")
  token = valid_data['Tokenizer'][k]
  token = ast.literal_eval(token)
  for i in range(len(id)):
    valid_lst.append([k,token[i],str(int(id[i]))])

In [ ]:
_NER_TAGS = ['0','1']

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_data = train_lst
train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"]
)


In [ ]:
eval_data = valid_lst
eval_data = pd.DataFrame(
    eval_data, columns=["sentence_id", "words", "labels"]
)


In [ ]:
eval_data

In [ ]:
ner_args = NERArgs()
ner_args.learning_rate = 1e-4
ner_args.train_batch_size = 32 
ner_args.use_multiprocessing = True 
ner_args.evaluate_during_training = True
ner_args.overwrite_output_di4r = True
ner_args.num_train_epochs = 30
ner_args.max_seq_length = 250
ner_args.save_model_every_epoch = False
#"camembert", "airesearch/wangchanberta-base-att-spm-uncased"
#"roberta", "papluca/xlm-roberta-base-language-detection"
model = NERModel(
     "camembert", "airesearch/wangchanberta-base-att-spm-uncased", args=ner_args, labels=_NER_TAGS
)

In [ ]:
model.train_model(
    train_data,
    eval_data=eval_data
)

In [ ]:
import matplotlib.pyplot as plt

training_loss = 
validation_loss =

epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'b', label='Training Loss')
plt.plot(epochs, validation_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
df = pd.read_excel("/content/Final_ner_test.xlsx")
test_lst = []
# trainset
for k in df.index:
  id = df['ID'][k].replace('[','').replace(']','').split(",")
  token = df['Tokenizer'][k]
  token = ast.literal_eval(token)
  for i in range(len(id)):
    test_lst.append([k,token[i],str(int(id[i]))])


  test_data = test_lst
test_data = pd.DataFrame(
    test_data, columns=["sentence_id", "words", "labels"]
)


In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
correct = 0
x=0
k=0
for i in range(len(preds_list)):
  for k in range(len(preds_list[i])):
    if (preds_list[i][k] == test_data['labels'][x]):
      correct+=1
    x+=1

print(correct/x)

In [ ]:
result, model_outputs, preds_list = model.eval_model(eval_data)

# Make predictions with the model
predictions, raw_outputs = model.predict(["ตู้เหล็กแบบ 2 บาน จำนวน 2 ตู้ โดยมีคุณลักษณะ ดังนี้ (1) มีมือจับชนิดผลัก (2) มีแผ่นชั้นปรับระดับ 3 ชิ้น (3) ฐานของตู้เหล็กมีความยาว 2 เมตร (4) มีระบบ noice system (5) สามารถกันนิวเคลียร์ได้ (6) มีลายกระหนก"])
print(predictions)